In [0]:
import requests
import sys
import xml.etree.ElementTree as ET
import json

In [0]:
#source = "https://www.lta.gov.sg/content/ltagov/en/map/fare-calculator.html"

mrtTripIndex = "https://www.lta.gov.sg/map/fareCalculator/mrtTripIndex.xml"
busTripIndex = "https://www.lta.gov.sg/map/fareCalculator/busTripIndex.xml"

trainAPI = "https://www.lta.gov.sg/content/ltagov/en/map/fare-calculator/jcr:content/map2-content/farecalculator.mrtget.html"
busAPI = "https://www.lta.gov.sg/content/ltagov/en/map/fare-calculator/jcr:content/map2-content/farecalculator.busget.html"

In [0]:
#mrt stuff

r = requests.get(mrtTripIndex)
root = ET.fromstring(r.content)
MRT_Keys = dict() #takes in either station names or station codes as index to spit out corresponding station id as value

for child in root:
    stationCodes = child.text.split('(')[1].strip(')').split('/')
    stationCodes = [i.strip() for i in stationCodes]

    MRT_Keys[child.text.split('(')[0].strip().upper()] = {
            'Station_Name':child.text.split('(')[0].strip(),
            'Station_Key':child.attrib['value'].split('_')[0],
            'Station_Codes':stationCodes,
        }


    for i in stationCodes:
        MRT_Keys[i.upper()] = {
            'Station_Name':child.text.split('(')[0].strip(),
            'Station_Key':child.attrib['value'].split('_')[0],
            'Station_Codes':stationCodes,
        }

MRT_fareTypes = {'adult':30,
                 'student':40,
                 'senior':39,
                 'workfare':38,
                 'disabilities':37}

def make_MRT_tripDetails(start,end,faretype): #translates start and end into stationKeys
    startKey = MRT_Keys.get(start.upper())
    endKey = MRT_Keys.get(end.upper())
    faretype = MRT_fareTypes.get(faretype.lower())
    if (startKey and endKey and faretype):#valid inputs
        return {'fare':faretype,'from':startKey['Station_Key'],'to':endKey['Station_Key']}
    else:
        return 'invalid inputs'

In [0]:
#bus stuff
r = requests.get(busTripIndex)
root = ET.fromstring(r.content)
bus_Keys = dict() #takes in bus no, stopCode and start/end of trip to generate stopKey
for busNo in root:
    stops = dict()
    for direction in busNo:
        for stop in direction:
            if stop.tag == 'busStop':
                stops[stop.text.split('-')[0].strip()] = {
                    'stop_Code':stop.text.split('-')[0].strip(),
                    'stop_Name':stop.text.split('-')[1].strip(),
                    'start_key':stop.attrib['id'].split('_')[0],
                    'end_key':stop.attrib['id'].split('_')[0],
                    }
    if len(list(busNo)) == 2:
        if len(list(busNo)[0])>0 and len(list(busNo)[1])>0:
        
            stops[direction[1].text.split('-')[0].strip()]['start_key'] = busNo[1][1].attrib['id'].split('_')[0]
            stops[direction[1].text.split('-')[0].strip()]['end_key'] = busNo[0][-1].attrib['id'].split('_')[0]
            stops[direction[-1].text.split('-')[0].strip()]['start_key'] = busNo[0][1].attrib['id'].split('_')[0]
            stops[direction[-1].text.split('-')[0].strip()]['end_key'] = busNo[1][-1].attrib['id'].split('_')[0]

    bus_Keys[busNo.attrib['id']] = stops

bus_fareTypes = {'adult':30,
                 'student':40,
                 'senior':39,
                 'workfare':38,
                 'disabilities':37}

def make_bus_tripDetails(busNo,start,end,faretype): #takes busNo, start and end stopCodes and translates into function params
    busObj = bus_Keys.get(str(busNo).upper())
    if busObj == None:
        return "invalid busNo"
    startKey = busObj.get(str(start))
    endKey = busObj.get(str(end))
    faretype = MRT_fareTypes.get(faretype.lower())
    if (startKey and endKey and faretype):#valid inputs
        return {'bus':str(busNo).upper(),'fare':faretype,'from':startKey['start_key'],'to':endKey['end_key']}
    else:
        return 'invalid inputs'

In [0]:
#Core function that calls the API for trip deets
def get_Next_Trip(mode,tripDetails,previousTrip=None):
    params = dict()
    if previousTrip:
        if previousTrip['tripNo']>=6:
            return 'limit of 6 trips exceeded!'
        tripNo = previousTrip['tripNo']+1
        params['addTripInfo'] = previousTrip['addTripInfo']
        params['tripInfo'] = previousTrip['tripInfo']
    else:
        tripNo = 1
        params['tripInfo'] = "usiAccumulatedDistance1=0-usiAccumulatedDistance2=0-usiAccumulatedDistance3=0-usiAccumulatedDistance4=0-usiAccumulatedDistance5=0-usiAccumulatedDistance6=0-usiAccumulatedFare1=0-usiAccumulatedFare2=0-usiAccumulatedFare3=0-usiAccumulatedFare4=0-usiAccumulatedFare5=0-usiAccumulatedFare6=0"
        params['addTripInfo'] = 0
    params.update(tripDetails)
    if mode.upper() == 'BUS':
        API = busAPI
    elif mode.upper() in ['MRT','TRAIN']:
        API = trainAPI
    else:
        return 'Invalid mode'
    r = requests.post(API,params=params)
    output = json.loads(r.content)
    output.update((('tripNo',tripNo),))
    return output

def calc_Fare_From_Dict(trip):
    # tripInfo contains history of previous fares as usiAccumulatedFare=123
    return sum([int(i.split('=')[1]) for i in trip['tripInfo'].split('-') if "Fare" in i])

In [0]:
def input_inputs():
    trips = []
    while len(trips)<6:
        mode = input('mode: ')
        if mode.upper() == 'BUS':
            busNo = input('Bus Number: ')
            start = input('Starting stop: ')
            end = input('Ending stop: ')
            trips.append({'busNo':busNo,'start':start,'end':end})
        elif mode.upper() in ['MRT','TRAIN']:
            start = input('Starting stop: ')
            end = input('Ending stop: ')
            trips.append({'start':start,'end':end})
        elif mode.upper() == " ":
            break
        else:
            print('mode can be ( bus / train / MRT )')
    return trips

def formField_inputs():
    tripInputs = []

    #@markdown TRIP 1 Inputs
    tempDict=dict()
    trip1_mode = "Bus" #@param ["Bus", "MRT","NIL"]
    if trip1_mode != "NIL":
        tempDict['mode'] = trip1_mode
        if trip1_mode == 'Bus':
            trip1_busNo = "225g" #@param {type:"string"}
            tempDict['busNo'] = trip1_busNo
        trip1_start = "84411" #@param {type:"string"}
        trip1_end = "84009" #@param {type:"string"}
        tempDict['start'] = trip1_start
        tempDict['end'] = trip1_end
        tripInputs.append(tempDict)

    #@markdown TRIP 2 Inputs
    tempDict=dict()
    trip2_mode = "MRT" #@param ["Bus", "MRT","NIL"]
    if trip2_mode != "NIL":
        tempDict['mode'] = trip2_mode
        if trip2_mode == 'Bus':
            trip2_busNo = " 1" #@param {type:"string"}
            tempDict['busNo'] = trip2_busNo
        trip2_start = "bedok" #@param {type:"string"}
        trip2_end = "buona vista" #@param {type:"string"}
        tempDict['start'] = trip2_start
        tempDict['end'] = trip2_end
        tripInputs.append(tempDict)

    #@markdown TRIP 3 Inputs
    tempDict=dict()
    trip3_mode = "Bus" #@param ["Bus", "MRT","NIL"]
    if trip3_mode != "NIL":
        tempDict['mode'] = trip3_mode
        if trip3_mode == 'Bus':
            trip3_busNo = "196" #@param {type:"string"}
            tempDict['busNo'] = trip3_busNo
        trip3_start = "11369" #@param {type:"string"}
        trip3_end = "19059" #@param {type:"string"}
        tempDict['start'] = trip3_start
        tempDict['end'] = trip3_end
        tripInputs.append(tempDict)

    #@markdown TRIP 4 Inputs
    tempDict=dict()
    trip4_mode = "NIL" #@param ["Bus", "MRT","NIL"]
    if trip4_mode != "NIL":
        tempDict['mode'] = trip4_mode
        if trip4_mode == 'Bus':
            trip4_busNo = "196" #@param {type:"string"}
            tempDict['busNo'] = trip4_busNo
        trip4_start = "wow" #@param {type:"string"}
        trip4_end = "wow" #@param {type:"string"}
        tempDict['start'] = trip4_start
        tempDict['end'] = trip4_end
        tripInputs.append(tempDict)

    #@markdown TRIP 5 Inputs
    tempDict=dict()
    trip5_mode = "NIL" #@param ["Bus", "MRT","NIL"]
    if trip5_mode != "NIL":
        tempDict['mode'] = trip5_mode
        if trip5_mode == 'Bus':
            trip5_busNo = "196" #@param {type:"string"}
            tempDict['busNo'] = trip5_busNo
        trip5_start = "wow" #@param {type:"string"}
        trip5_end = "wow" #@param {type:"string"}
        tempDict['start'] = trip5_start
        tempDict['end'] = trip5_end
        tripInputs.append(tempDict)

    #@markdown TRIP 6 Inputs
    tempDict=dict()
    trip6_mode = "NIL" #@param ["Bus", "MRT","NIL"]
    if trip6_mode != "NIL":
        tempDict['mode'] = trip6_mode
        if trip6_mode == 'Bus':
            trip6_busNo = "196" #@param {type:"string"}
            tempDict['busNo'] = trip6_busNo
        trip6_start = "wow" #@param {type:"string"}
        trip6_end = "wow" #@param {type:"string"}
        tempDict['start'] = trip6_start
        tempDict['end'] = trip6_end
        tripInputs.append(tempDict)
    return tripInputs

In [0]:
def main(trips,faretype):
    previousTrip = None
    n = 1
    for trip in trips:
        if trip['mode'] == 'Bus':
            print(f"Trip {n} on bus {trip['busNo']} from {trip['start']} to {trip['end']}")
            tripDetails = make_bus_tripDetails(trip['busNo'],trip['start'],trip['end'],faretype)
            previousTrip = get_Next_Trip('bus',tripDetails,previousTrip=previousTrip)
        else:
            print(f"Trip {n} from {trip['start']} to {trip['end']}")
            tripDetails = make_MRT_tripDetails(trip['start'],trip['end'],faretype)
            previousTrip = get_Next_Trip('mrt',tripDetails,previousTrip=previousTrip)
        #print(previousTrip)
        n+=1
    return calc_Fare_From_Dict(previousTrip)
print(main(formField_inputs(),'adult'))
#print(main(input_inputs(),'adult'))

Trip 1 on bus 225g from 84411 to 84009
Trip 2 from bedok to buona vista
Trip 3 on bus 196 from 11369 to 19059
193
